In [ ]:
#Tutorial Followed: https://huggingface.co/blog/fine-tune-wav2vec2-english

# !pip install datasets>=1.18.3
# !pip install transformers==4.41.2
# !pip install librosa
# !pip install jiwer
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !pip install wandb
# !pip install pandas

from datasets import Dataset, load_dataset, load_metric, load_from_disk, concatenate_datasets
import pandas as pd
from datasets import Audio, DatasetDict

In [ ]:
# dataset = load_dataset("parler-tts/mls_eng_10k", split="train", streaming=True)
# dataset = dataset.shuffle(seed=42, buffer_size=10_000)
# dataset = dataset.take(3000)

# from functools import partial
# from datasets import Dataset

# def gen_from_iterable_dataset(iterable_ds):
#     yield from iterable_ds

# ds = Dataset.from_generator(partial(gen_from_iterable_dataset, dataset), features=dataset.features)

# print(ds[0]['audio'])

# ds.save_to_disk("mls_3k_sample")

In [ ]:
# ## we will load the both of the data here.
# train_df = pd.read_excel("train_audio_normalized_text_pair.xlsx")
# test_df = pd.read_excel("val_audio_normalized_text_pair.xlsx")

# ## we will rename the columns as "audio", "sentence".
# train_df.columns = ["audio", "transcript"]
# test_df.columns = ["audio", "transcript"]

# train_audio_path = './train_audios/'
# test_audio_path = './dev_audios/'

# train_df['audio'] = train_audio_path + train_df['audio'].astype(str)
# test_df['audio'] = test_audio_path + test_df['audio'].astype(str)

# ## convert the pandas dataframes to dataset 
# train_dataset = Dataset.from_pandas(train_df)
# test_dataset = Dataset.from_pandas(test_df)

# ## convert the sample rate of every audio files using cast_column function
# train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
# test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16000))

# dataset = DatasetDict({"train":train_dataset,"test":test_dataset})

# print(dataset)

In [ ]:
dataset = load_from_disk('mls_3k_sample', keep_in_memory = True)

print(dataset)
print(type(dataset[0]['audio']))
print(dataset[0]['audio'])

# dataset = dataset.shuffle(seed=42).select(range(1100))

dataset = dataset.train_test_split(test_size=0.1)
print(dataset)

dataset = dataset.remove_columns(['original_path', 'begin_time', 'end_time','audio_duration', 'speaker_id', 'book_id'])

In [ ]:
# hs_dataset = load_from_disk('mls_3k_sample', keep_in_memory = True)
# hs_dataset = hs_dataset.remove_columns(['original_path', 'begin_time', 'end_time','audio_duration', 'speaker_id', 'book_id'])
# hs_dataset = hs_dataset.shuffle(seed=42).select(range(600))


# train_df = pd.read_excel("train_audio_normalized_text_pair.xlsx")
# train_df.columns = ["audio", "transcript"]
# train_audio_path = './train_audios/'
# train_df['audio'] = train_audio_path + train_df['audio'].astype(str)
# train_dataset = Dataset.from_pandas(train_df)

# ## convert the sample rate of every audio files using cast_column function
# train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
# train_dataset = train_dataset.shuffle(seed=42).select(range(6000))

# hs_dataset = hs_dataset.cast_column("audio", Audio(sampling_rate=16000))

# dataset = DatasetDict({"train":train_dataset,"test":hs_dataset})

# print(dataset)

In [ ]:
# hs_dataset = load_from_disk('mls_3k_sample', keep_in_memory = True)
# hs_dataset = hs_dataset.remove_columns(['original_path', 'begin_time', 'end_time','audio_duration', 'speaker_id', 'book_id'])
# hs_dataset = hs_dataset.shuffle(seed=42).select(range(1300))


# train_df = pd.read_excel("train_audio_normalized_text_pair.xlsx")
# train_df.columns = ["audio", "transcript"]
# train_audio_path = './train_audios/'
# train_df['audio'] = train_audio_path + train_df['audio'].astype(str)
# train_dataset = Dataset.from_pandas(train_df)

# ## convert the sample rate of every audio files using cast_column function
# train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
# train_dataset = train_dataset.shuffle(seed=42).select(range(1000))

# hs_dataset = hs_dataset.cast_column("audio", Audio(sampling_rate=16000))
# hs_dataset = hs_dataset.train_test_split(test_size=0.2)

# concat_train_ds = concatenate_datasets([train_dataset, hs_dataset['train']])

# dataset = DatasetDict({"train":concat_train_ds,"test":hs_dataset['test']})

# print(dataset)

In [ ]:
import re
chars_to_ignore_regex = r'[!@#$%^&*()_+\-=\[\]{};\":\\|,.<>\/?`~]'

def remove_special_characters(batch):
    batch["transcript"] = re.sub(chars_to_ignore_regex, '', batch["transcript"]).lower()
    return batch


dataset = dataset.map(remove_special_characters)

def extract_all_chars(batch):
  all_text = " ".join(batch["transcript"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names["train"])

In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
# import IPython.display as ipd
# import numpy as np
# import random

# rand_int = random.randint(0, len(dataset["train"]))

# print(dataset["train"][rand_int]["transcript"])
# ipd.Audio(data=np.asarray(dataset["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)
# rand_int = random.randint(0, len(dataset["train"]))

# print("Target text:", dataset["train"][rand_int]["transcript"])
# print("Input array shape:", np.asarray(dataset["train"][rand_int]["audio"]["array"]).shape)
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)# print("Sampling rate:", dataset["train"][rand_int]["audio"]["sampling_rate"])

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcript"]).input_ids
    return batch

In [ ]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"])

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
import numpy as np

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)


In [ ]:
model.freeze_feature_extractor()



In [ ]:
from transformers import TrainingArguments

output_path = 'finetunedWav2Vec_hs_3k'

training_args = TrainingArguments(
  output_dir= output_path + '/model',
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=20,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

print(training_args.device)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

In [ ]:
processor.save_pretrained(output_path + '/processor')

In [ ]:
device = torch.device('cuda:0')
repo_name = './finetunedWav2Vec_hs_3k/'

eval_model = Wav2Vec2ForCTC.from_pretrained(repo_name+"model/checkpoint-6000").to(device)
eval_processor = Wav2Vec2Processor.from_pretrained(repo_name+"processor")

def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = eval_model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = eval_processor.batch_decode(pred_ids)[0]
  batch["text"] = eval_processor.decode(batch["labels"], group_tokens=False)

  return batch

results = dataset["test"].map(map_to_result, remove_columns=dataset["test"].column_names)

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(results)

In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(dataset["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))